# Wstęp do sztucznej inteligencji
Ćwiczenie 4 - Drzewa Decyzyjne  
Maciej Łodziński  

Celem ćwiczenia była implementacja drzewa decyzyjnego ID3 z uwzględnieniem maksymalnej głębokości przeszukiwania.   
Zbiorem danych, na którym trzeba było testować wyliczony model są dane pacjentów cierpiących lub nie na chorobę   
serca znajdujące się w pliku `cardio_train.csv`.

In [67]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
from random import choices
from random import randint

In [68]:
#clean split data into train, validation and test data -> 6:2:2
def split_data(filename, deli):
    data = pd.read_csv(filename, sep=deli)
    data = data.dropna(axis=0)
    data = data.head(10000)
    train_data, validation_and_test_data = train_test_split(data, test_size = 0.4)
    validation_data, test_data = train_test_split(validation_and_test_data, test_size = 0.5)
    return train_data.reset_index(), validation_data.reset_index(), test_data.reset_index()

### Dyskretyzacja danych
Aby zapobiec nadmiernym dopasowaniu się drzewa dezycyjnego do danych trenujących dzielę je  
na kategorie tj. dyskretyzuję. Każdą cechę, która tego potrzebuje dzielę na kilka przedziałów liczbowych.

In [69]:
def discretize_data(df):
    df = df.drop(['index', 'id'], axis=1)
    df['age'] = pd.cut(x=df['age'], bins=8, labels=["1-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71-80"])
    df['height'] = pd.cut(x=df['height'], bins=[0, 155, 170, 185, 220], labels=['short', 'medium', 'high', 'higher'])
    df['weight'] = pd.cut(x=df['weight'], bins=[0, 50, 60, 70, 80, 90, 100, 150, 400],
                          labels=['light', '51-60', '61-70', '71-80','81-90','91-100','101-150','151-400'])
    df['ap_hi'] = pd.cut(x=df['ap_hi'], bins=[0, 50, 90, 120, 150, 200], labels=['lower', 'low', 'medium', 'high', 'very_high'])
    return df

def split_and_disc(filename, deli):
    data = split_data(filename, deli)
    new_data = []
    for data_set in data:
        new_data.append(discretize_data(data_set))
    return new_data

### Implementacja interfejsu Solver

In [70]:
class Solver():
    def __init__(self, max_depth, min_samples=0):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.decision_tree = None
    
    def get_parameters(self):
        return {"max_depth" : self.max_depth, "min_samples" : self.min_samples}
    
    def fit(self, X, y):
        decision_tree = DecisionTree(self.max_depth)
        train_data = pd.concat([X, y], axis=1)        
        decision_tree.id3(train_data)
        self.decision_tree = decision_tree
        
    def predict(self, X):
        if self.decision_tree == None: return
        predictions = []
        for instance in X:
            predition = self.decision_tree.predict(decision_tree.tree, instance)
            predictions.append(predition)
        return predictions

### Implementacja drzewa decyzyjnego
- Główną funckją drzewa jest `make_tree`, które wywoływane jest rekurencyjnie, do momentu dojścia do maksymalnej głębokości  
  lub gdy dane w *węźle* będą już tylko jednego typu. W przypadku niedokończenia drzewa przez ogranicznie `max_depth`,  
  model wybiera najczęściej występującą klasę. Samo drzewo reprezentowane jest przez słownik.  
- Funkcja `create_sub_tree` tworzy podrzewo na podstawie danej cechy i usuwa przypadki tej cechy z danych.
- Funkcja `best_feature` zwraca cechę, na podstawie której będzie rosnąć drzewo w następnej kolejności tj. tą która ma  
  największy przyrost informacyjny.

In [71]:
class DecisionTree:
    def __init__(self, max_depth, min_samples=0):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None
        
    def id3(self, train_data):
        label =  train_data.columns[-1]
        tree = {}
        unique_outputs = train_data[label].unique()
        self.make_tree(tree, None, train_data, unique_outputs)
        self.tree = tree
    
    def make_tree(self, root, prev_feature_value, train_data, unique_outputs, depth=0):
        label =  train_data.columns[-1]
        if train_data.shape[0] > self.min_samples:
            feature = self.best_feature(train_data, unique_outputs)
            tree, train_data = self.create_sub_tree(feature, train_data, unique_outputs)

            if prev_feature_value != None:
                root[prev_feature_value] = {}
                root[prev_feature_value][feature] = tree
                next_root = root[prev_feature_value][feature]
            else:
                root[feature] = tree
                next_root = root[feature]

            for node, branch in list(next_root.items()):
                if depth > self.max_depth and train_data.shape[0] > self.min_samples: #check if depth in bounds
                    next_root[node] = most_common(train_data)
                    return
                    
                elif branch == None: #check if node isn't terminal
                    feature_value_data = train_data[train_data[feature] == node]
                    self.make_tree(next_root, node, feature_value_data, unique_outputs, depth+1)
                    
    def create_sub_tree(self, feature_name, train_data, unique_outputs):
        label =  train_data.columns[-1]
        feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
        tree = {}
        
        for fv, count in feature_value_count_dict.iteritems():
            fv_data = train_data[train_data[feature_name] == fv]
            assigned_to_node = False
            for output in unique_outputs:
                class_count = fv_data[fv_data[label] == output].shape[0]
                if class_count == count:
                    tree[fv] = output
                    train_data = train_data[train_data[feature_name] != fv]
                    assigned_to_node = True
            if not assigned_to_node:
                tree[fv] = None #unfinished node
        return tree, train_data
    
    def best_feature(self, train_data, unique_outputs):
        label =  train_data.columns[-1]
        feature_list = train_data.columns.drop(label)
        max_info_gain = -1
        max_info_feature = None

        for feature in feature_list:
            feature_info_gain = inf_gain(feature, train_data, unique_outputs)
            if max_info_gain < feature_info_gain:
                max_info_gain = feature_info_gain
                max_info_feature = feature
        return max_info_feature


### Utils - funkcje pomocnicze

In [72]:
def inf_gain(feature_name, train_data, unique_outputs):
    label =  train_data.columns[-1]
    feature_value_list = train_data[feature_name].unique()
    num_rows = train_data.shape[0]
    feature_info = 0

    for fv in feature_value_list:
        fv_data = train_data[train_data[feature_name] == fv]
        fv_count = fv_data.shape[0]
        fv_entropy = feature_entropy(fv_data, unique_outputs)
        fv_probability = fv_count / num_rows
        feature_info += fv_probability * fv_entropy
    return total_entropy(train_data, unique_outputs) - feature_info

def total_entropy(train_data, unique_outputs):
    label =  train_data.columns[-1]
    num_rows = train_data.shape[0]
    entropy = 0
    for output in unique_outputs:
        output_count = train_data[train_data[label] == output].shape[0]
        output_entropy = - (output_count/num_rows) * np.log2(output_count / num_rows)
        entropy += output_entropy
    return entropy

def feature_entropy(feature_value_data, unique_outputs):
    label =  train_data.columns[-1]
    output_count = feature_value_data.shape[0]
    entropy = 0
    for output in unique_outputs:
        label_class_count = feature_value_data[feature_value_data[label] == output].shape[0] 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count / output_count
            entropy_class = - probability_class * np.log2(probability_class)
        entropy += entropy_class
    return entropy

def most_common(train_data):
    label =  train_data.columns[-1]
    labels = train_data[label]
    unique, pos = np.unique(labels, return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()

    return unique[maxpos]

def predict(tree, instance):
    if not isinstance(tree, dict):
        return tree
    else:
        root_node = next(iter(tree))
        feature_value = instance[root_node]
        if feature_value in tree[root_node]:
            return predict(tree[root_node][feature_value], instance)
        else:
            return None

def acuracy(tree, test_data):
    label =  train_data.columns[-1]
    correct = 0
    wrong = 0
    for index, _ in test_data.iterrows():
        instance = test_data.iloc[index]
        result = predict(tree, instance)
        if result == test_data[label].iloc[index]:
            correct += 1
        else:
            wrong += 1
    accuracy = correct / (correct + wrong)
    return accuracy

def precision_and_recall(tree, validation_data):
    label =  train_data.columns[-1]
    size = validation_data.shape[0]
    res = {'TP':0, 'FP':0, 'FN':0, 'TN':0}
    outputs = validation_data[label]

    for index, _ in validation_data.iterrows():
        instance = validation_data.iloc[index]
        result = predict(tree, instance)
        if result == 1 and outputs[index] == 1:
             res['TP'] += 1
        elif result == 1 and outputs[index] == 0:
             res['FP'] += 1
        elif result == 0 and outputs[index] == 1:
            res['FN'] += 1
        elif result == 0 and outputs[index] == 0:
            res['TN'] += 1

    precision = res['TP']/(res['TP'] + res['FP'])
    recall = res['TP']/(res['TP'] + res['FN'])
    return precision, recall


### Testy
Postanowiono przetestować zbiór walidujący danych. Na początku zmniejszono liczbę próbek  
danych do 10000 w celu przyśpieszenia czasu symulacji a następnie testowano model dla kilku  
wartości maksymalnej głębokości przeszukiwania od 1 do 12. Jakość oceniono na postawie dokładności,  
precyzji oraz pełności. Poniżej przedstawione zostały wyniki tych głębokości.

In [73]:
# testing max_depth parameter
train_data, validation_data, test_data = split_and_disc("cardio_train.csv", ';')

for depth in range(1, 13):
    solver = Solver(depth)
    X = train_data.iloc[:,:-1]
    y = train_data.iloc[:,-1]
    
    solver.fit(X, y)
    decision_tree = solver.decision_tree
    accuracy = acuracy(decision_tree.tree, validation_data)
    precision, recall = precision_and_recall(decision_tree.tree, validation_data)
    
    print(f'MaxDepth = {depth}:\n    -> Accuracy = {round(accuracy*100, 2)}%\n\
    -> Precision = {round(precision*100, 2)}%\n\
    -> Recall = {round(recall*100, 2)}%\n')

MaxDepth = 1:
    -> Accuracy = 27.7%
    -> Precision = 51.32%
    -> Recall = 54.78%

MaxDepth = 2:
    -> Accuracy = 28.9%
    -> Precision = 62.16%
    -> Recall = 59.47%

MaxDepth = 3:
    -> Accuracy = 39.4%
    -> Precision = 66.34%
    -> Recall = 70.59%

MaxDepth = 4:
    -> Accuracy = 50.2%
    -> Precision = 66.67%
    -> Recall = 67.18%

MaxDepth = 5:
    -> Accuracy = 52.35%
    -> Precision = 65.29%
    -> Recall = 60.92%

MaxDepth = 6:
    -> Accuracy = 51.6%
    -> Precision = 64.1%
    -> Recall = 63.38%

MaxDepth = 7:
    -> Accuracy = 55.35%
    -> Precision = 65.24%
    -> Recall = 62.41%

MaxDepth = 8:
    -> Accuracy = 49.6%
    -> Precision = 65.79%
    -> Recall = 66.12%

MaxDepth = 9:
    -> Accuracy = 45.45%
    -> Precision = 65.75%
    -> Recall = 67.79%

MaxDepth = 10:
    -> Accuracy = 40.35%
    -> Precision = 65.42%
    -> Recall = 70.71%

MaxDepth = 11:
    -> Accuracy = 40.35%
    -> Precision = 65.42%
    -> Recall = 70.71%

MaxDepth = 12:
    -> Accu

### Wnioski z eksperymentu
Z przeprowadzonych wyżej eksperymentów można wysnuć następujące wnioski:  

Gdy `max_depth` przekroczy 7 oceny jakości przestają rosnąć, a nawet zaczynają maleć co widać miedzy przypadkami `max_depth 7->8`. Na ogół im większa maksymalna głębokość drzewa tym model jest dokładniejszy - jest o wiele bardziej  dopsasowany do danych trenujących i estymata wyjścia jest lepsza. Jednak przy zbyt dużych głębokościach obserwujemy  zmniejszenie ocen jakości. Dzieje  się tak, gdyż drzewa decyzyjne są bardzo podatne na przetrenowanie. To co się widzimy to  nadmierne dopasowanie się drzewa do  zbioru trenującego, które potem skutkuje marnymi wynikami na zbiorze walidacyjnym. Ponadto  wyniki mogą nie być najlepsze przez same dane lub dyskretyzację tych danych. 

### Dobór max_depth
W przypadku naszego problemu - wykrywanie choroby serca, bardziej zależy nam na pełności niż na precyzji, gdyż  
chcemy zaminimalizować liczbę sytuacji, w których nie wykryjemy choroby serca u chorego pacjenta. W związku z tym,  
ostatecznie dobrano wartość parametru `max_depth = 7`, ponieważ ten przypadek łączy w sobie prawie najlepsze możliwe  
wyniki ocen jakości i nie zabiera dużo czasu.  

### Wnioski ogólne
Skuteczność modelu drzewa decyzyjnego zależy od wielu czynników:  
- samych danych.  
- sposobie podziału zbioru danych na podzbiory treningowy, walidacyjny, testowy.  
- doboru parametrów modelu przy użyciu zbioru walidacyjnego  
- rozmiaru danych wejściowych - im więcej danych tym dokładniejszy model otrzymamy.  
- liczby atrybutów zbioru danych - przy bardzo dużej ilości atrybutów wygenerowany model może działać niepoprawnie,  
  lub jego  generacja moze trwać bardzo długo.
- wadą jest jego długotrwałe wyliczanie 
- zaletą jest szybka klasyfikacja nowych danych na podstawie już raz wyliczonego modelu 